In [31]:
!pip install mlflow
!pip install xgboost


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.xgboost
import xgboost as xgb
import matplotlib.pyplot as plt
import os

In [6]:
PROCESSED_DIR = "../data/processed"
MODEL_DIR = "../data/models/path_predictor"
os.makedirs(MODEL_DIR, exist_ok=True)

In [7]:
student_info = pd.read_csv(f"{PROCESSED_DIR}/student_info_normalized.csv")
courses = pd.read_csv(f"{PROCESSED_DIR}/courses_normalized.csv")
registrations = pd.read_csv(f"{PROCESSED_DIR}/registrations_normalized.csv")
vle_info = pd.read_csv(f"{PROCESSED_DIR}/vle_info_normalized.csv")
student_vle = pd.read_csv(f"{PROCESSED_DIR}/student_vle_normalized.csv")
assessments = pd.read_csv(f"{PROCESSED_DIR}/assessments_normalized.csv")
student_assessment = pd.read_csv(f"{PROCESSED_DIR}/student_assessment_normalized.csv")

print("✔️ Chargement terminé !")

✔️ Chargement terminé !


In [17]:
student_vle = student_vle.rename(columns={"id_student": "student_id"})

# 1) Activité VLE : total clicks + moyenne + variance
vle_features = student_vle.groupby("student_id").agg({
    "sum_click": ["sum", "mean", "std"]
})
vle_features.columns = ["click_total", "click_mean", "click_std"]
vle_features = vle_features.fillna(0).reset_index()

# 2) Notes des évaluations
score_features = student_assessment.groupby("student_id").agg({
    "score": ["mean", "std", "min", "max"]
})
score_features.columns = ["score_mean", "score_std", "score_min", "score_max"]
score_features = score_features.fillna(0).reset_index()

# 3) Fusion avec student_info
df = student_info.merge(vle_features, on="student_id", how="left")
df = df.merge(score_features, on="student_id", how="left")

df = df.fillna(0)

print("✔️ Features créées :", df.shape)


✔️ Features créées : (32593, 19)


In [19]:
# LABEL : réussite ou échec
# On utilise 'final_result' dans student_info (Pass / Fail)
df["target"] = df["final_result"].apply(lambda x: 1 if x.lower() == "pass" else 0)

# Drop colonnes inutiles pour le modèle
df_model = df.drop(columns=["final_result", "gender", "region", "student_id"])

# Séparation features / target
X = df_model.drop(columns=["target"])
y = df_model["target"]

print("✔️ Data prête pour le modèle :", X.shape, y.shape)


✔️ Data prête pour le modèle : (32593, 15) (32593,)


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("✔️ Split terminé")

✔️ Split terminé


In [26]:
import xgboost as xgb
import mlflow
import mlflow.xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd

# -------------------------
# 1️⃣ Répertoire de sauvegarde
# -------------------------
MODEL_DIR = "../models"
os.makedirs(MODEL_DIR, exist_ok=True)

# -------------------------
# 2️⃣ Colonnes catégorielles et numériques
# -------------------------
categorical_cols = ["code_module", "code_presentation", "highest_education",
                    "imd_band", "age_band", "disability"]

numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# -------------------------
# 3️⃣ Préparation des données
# -------------------------
# Colonnes catégorielles : remplir NaN et encoder avec LabelEncoder
for col in categorical_cols:
    X[col] = X[col].fillna("Unknown")
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Colonnes numériques : remplacer NaN par 0
X[numeric_cols] = X[numeric_cols].fillna(0)

# Vérification
print("Types de colonnes après préparation :")
print(X.dtypes)

# -------------------------
# 4️⃣ Split train/test
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -------------------------
# 5️⃣ Configuration MLflow
# -------------------------
mlflow.set_tracking_uri("file:../mlruns")  # suivi local
mlflow.set_experiment("PathPredictor")

with mlflow.start_run():
    
    # -------------------------
    # 6️⃣ Définition du modèle
    # -------------------------
    model = xgb.XGBClassifier(
        n_estimators=150,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic"
    )
    
    # -------------------------
    # 7️⃣ Entraînement
    # -------------------------
    model.fit(X_train, y_train)
    
    # -------------------------
    # 8️⃣ Log MLflow
    # -------------------------
    mlflow.xgboost.log_model(model, "path_predictor_model")
    mlflow.log_params(model.get_params())
    
    # -------------------------
    # 9️⃣ Évaluation
    # -------------------------
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", acc)
    
    print(f"✔️ Accuracy du modèle : {acc:.4f}")
    
    # -------------------------
    # 🔟 Sauvegarde locale du modèle
    # -------------------------
    model.save_model(f"{MODEL_DIR}/path_predictor.json")
    print("✔️ Modèle sauvegardé dans :", MODEL_DIR)


Types de colonnes après préparation :
code_module               int32
code_presentation         int32
highest_education         int32
imd_band                  int32
age_band                  int32
num_of_prev_attempts      int64
studied_credits           int64
disability                int32
click_total             float64
click_mean              float64
click_std               float64
score_mean              float64
score_std               float64
score_min               float64
score_max               float64
dtype: object


2025/12/09 18:46:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✔️ Accuracy du modèle : 0.8032
✔️ Modèle sauvegardé dans : ../models


In [28]:
def predict_student(student_id, module_code):
    # Vérifier que l'étudiant existe
    if student_id not in df["student_id"].values:
        print("❌ Étudiant introuvable")
        return
    
    # Filtrer les données de l'étudiant
    x = df[df["student_id"] == student_id].copy()
    
    # Ajouter le module à prédire (si différent de celui déjà dans df)
    x["code_module"] = module_code
    
    # Colonnes à supprimer avant prédiction
    drop_cols = ["final_result", "gender", "region", "student_id", "target"]
    x = x.drop(columns=[c for c in drop_cols if c in x.columns])
    
    # ⚠️ Refaire le LabelEncoding sur les colonnes catégorielles
    categorical_cols = ["code_module", "code_presentation", "highest_education",
                        "imd_band", "age_band", "disability"]
    for col in categorical_cols:
        if col in x.columns:
            x[col] = x[col].fillna("Unknown")
            le = LabelEncoder()
            x[col] = le.fit_transform(x[col].astype(str))  # même traitement que l'entraînement
    
    # Colonnes numériques : remplacer NaN
    numeric_cols = x.select_dtypes(include=["int64", "float64"]).columns.tolist()
    x[numeric_cols] = x[numeric_cols].fillna(0)
    
    # Prédiction
    proba = model.predict_proba(x)[0][1]  # probabilité de réussite
    print(f"🎯 Probabilité de réussite : {proba*100:.2f}%")
    
    # Alertes de risque
    if proba < 0.4:
        print("⚠️ RISQUE ÉLEVÉ — intervention recommandée")
    elif proba < 0.6:
        print("⚠️ Risque moyen — suivre la progression")
    else:
        print("✔️ Bon niveau — progression normale")


In [30]:
# Étudiant et module à prédire
predict_student(student_id=11391, module_code="BBB")
predict_student(student_id=58873, module_code="AAA")


🎯 Probabilité de réussite : 52.27%
⚠️ Risque moyen — suivre la progression
🎯 Probabilité de réussite : 89.04%
✔️ Bon niveau — progression normale
